# 每十个百分点召回下的准确率

In [1]:
#todo

# 将概率换算成信用分

In [1]:
def score_card_cal(pred,intercept=700,pred_cut=0.01,pdo=25):
    import numpy as np
    B = pdo/np.log(2)
    A = intercept + B*log(pred_cut/(1-pred_cut))
    return  int(A-B*np.log(pred/(1-pred)))

# 计算AUC 绘制ROC曲线

In [1]:
from sklearn.metrics import roc_auc_score
# 直接使用sklearn就可以

In [ ]:
#ROC曲线
def Roc_Curve(y_true,y_pre,have_auc=False):
    '''df as gbie p'''
    from sklearn import metrics
    fpr_lst,tpr_lst=metrics.roc_curve(y_true,y_pre)[:2]
    auc=metrics.auc(fpr_lst,tpr_lst)
    roc_curve=pd.DataFrame(np.array([tpr_lst,fpr_lst]).T,columns=["Sensitivity","1-Specificity"])
    roc_curve=roc_curve.drop_duplicates("1-Specificity")
    roc_curve.plot('1-Specificity','Sensitivity',\
                   title="ROC Curve \n AUC=%s" % str(auc))
    if have_auc:
        return auc,roc_curve

# 计算KS值 并绘图

In [ ]:
#KS曲线
def KS(y_true,y_pre,group_sep=0.01):
    from copy import deepcopy
    import matplotlib.pyplot as plt
    plt.style.use('ggplot')
    '''df as gbie p'''
    DfIn=pd.DataFrame([y_true,y_pre]).T
    DfIn.columns=['gbie','p']
    DfIn=DfIn.sort_values('p',ascending=False)
    length=len(DfIn)
    group=pd.cut(list(range(length)),bins=1.0/group_sep,labels=np.arange(0+group_sep,1+group_sep,group_sep))
    DfIn['group']=list(group)
    length_1=sum(DfIn.gbie)
    length_0=length-length_1
    cum_0=[]
    cum_1=[]
    for i in sorted(set(group)):
        cum_tmp=DfIn.loc[DfIn.group<=i,'gbie']
        cum_0.append(float(len(cum_tmp)-sum(cum_tmp))/length_0)
        cum_1.append(float(sum(cum_tmp))/length_1)
    DfPlot=pd.DataFrame(np.array([sorted(list(set(group))),cum_0,cum_1]).T, 
                        columns=['Percent','Good','Bad'])
    Ks=max(DfPlot.Bad-DfPlot.Good)
    argmax=DfPlot.loc[(DfPlot.Bad-DfPlot.Good).argmax(),'Percent']
    DfPlot['ks']=DfPlot.Bad-DfPlot.Good
    DfPlot.plot('Percent',['Good','Bad','ks'],\
                title='KS Curve\n ks=%.3f and cut_off=%.2f' % (Ks,argmax))
    return plt,Ks